# Project 1

Authored by Joshua Burrows and Maksim Nikiforov.

## Introduction

To do (Maksim).

In [3]:
import pandas as pd

# Read in the data
bike_data = pd.read_csv("bikeDetails.csv")

# Display a small snippet of it
bike_data.head()

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN


## Exploratory Data Analysis

## Prediction of `selling_price`